# Moderated Chatbot

We will now combine templating and content filtering in a chatbot. Additionally we will make use of the orchestration services history capability.

We start with building the chatbot class. This class will be responsible for providing the template for the user query and managing the history. Note that the `chat` method uses the `history` parameter to send along the history to the orchestration service.

In [ ]:
from typing import List

from gen_ai_hub.orchestration.models.message import Message, SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.service import OrchestrationService


class OrchestrationChatbot:
    def __init__(self, orchestration_service: OrchestrationService):
        self.service = orchestration_service
        self.service.config.template = Template(
            messages=[
                SystemMessage("You are a helpful chatbot assistant."),
                UserMessage("{{?user_query}}"),
            ],
        )
        self.history: List[Message] = []

    def chat(self, user_input):
        response = self.service.run(
            template_values=[
                TemplateValue(name="user_query", value=user_input),
            ],
            history=self.history,
        )

        message = response.orchestration_result.choices[0].message

        self.history = response.module_results.templating
        self.history.append(message)

        return message.content

Next up we will create an orchestration configuration. Next to the model parameter we also pass a content filter for input and output to ensure a nice conversation.

In [ ]:
from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.content_filter import AzureContentFilter
from gen_ai_hub.orchestration.models.llm import LLM

content_filter = AzureContentFilter(
    hate=0,
    sexual=0,
    self_harm=0,
    violence=0,
)
config = OrchestrationConfig(
    llm=LLM(name="gemini-1.5-flash"),
    template=None,
    input_filters=[content_filter],
    output_filters=[content_filter],
)

orchestration_service = OrchestrationService(
    api_url=os.environ["AICORE_ORCHESTRATION_DEPLOYMENT_URL"],
    config=config,
)

Lets chat with our bot and test if it is able to recall a conversation!

In [ ]:
bot = OrchestrationChatbot(orchestration_service=orchestration_service)

In [ ]:
print(bot.chat("Hello, how are you?"))

In [ ]:
print(bot.chat("What's the weather like today?"))

In [ ]:
print(bot.chat("Can you remember what I first asked you?"))

# Summary

In this exercise you got hands-on experience on how to use orchestration in complex scenarios. This exercise concludes the SAP generative AI hub SDK (Python) track. Feel free to futher explore the orchestration capabilities using this SDK or checkout the [SAP Cloud SDK for AI (JavaScript)](tbd) track.